In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
from IPython.core.display import display, HTML
import pickle

C:\Users\nekna\AppData\Local\Temp\ipykernel_13772\1480811194.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# EDA

Need to predict house prices

In [2]:
os.chdir('..')
train = pd.read_csv('data/raw/train.csv')
test = pd.read_csv('data/raw/test.csv')

# Preprocessing

In [3]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

We drop target column to exclude in from model and have an inference on it

In [4]:
train = train.set_index('Id')
test = test.set_index('Id')


Target_cols = ['SalePrice']
Target = train[Target_cols]
train.drop(Target_cols, axis=1, inplace=True)
train_copy = train.copy()
Target

,SalePrice
Id,
1,208500
2,181500
3,223500
4,140000
5,250000
...,...
1456,175000
1457,210000
1458,266500


In [5]:
int_cols = [
    'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
    '1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea' ,'BsmtFullBath','BsmtHalfBath','FullBath' ,'HalfBath','BedroomAbvGr' ,'KitchenAbvGr',
    'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF',  'OpenPorchSF',  'EnclosedPorch',  '3SsnPorch',  'ScreenPorch',
    'PoolArea',  'MiscVal',  'MoSold','YrSold'
    ]
float_cols = [
    'LotFrontage', 'MasVnrArea', 'GarageYrBlt'
]
categorial_cols = train.columns.drop(int_cols).drop(float_cols).to_list()
test_copy = test.copy()
def preprocessing(df:pd.DataFrame): 
    df[int_cols] = df[int_cols].fillna(0).astype(np.int16)
    df[float_cols] = df[float_cols].fillna(0.0).astype(np.float32)    
    df[categorial_cols] = df[categorial_cols].fillna('None').astype('category')
    return df

train_copy = preprocessing(train_copy)
test_copy = preprocessing(test_copy)
display(HTML(train_copy.head().to_html()))
display(HTML(test_copy.head().to_html()))


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal
2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal
3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal
4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468,LwQ,144,270,882,GasA,TA,Y,SBrkr,896,0,0,896,0,0,1,0,2,1,TA,5,Typ,0,None,Attchd,1961.0,Unf,1,730,TA,TA,Y,140,0,0,0,120,0,None,MnPrv,None,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923,Unf,0,406,1329,GasA,TA,Y,SBrkr,1329,0,0,1329,0,0,1,1,3,1,Gd,6,Typ,0,None,Attchd,1958.0,Unf,1,312,TA,TA,Y,393,36,0,0,0,0,None,None,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791,Unf,0,137,928,GasA,Gd,Y,SBrkr,928,701,0,1629,0,0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2,482,TA,TA,Y,212,34,0,0,0,0,None,MnPrv,None,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602,Unf,0,324,926,GasA,Ex,Y,SBrkr,926,678,0,1604,0,0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2,470,TA,TA,Y,360,36,0,0,0,0,None,None,None,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,None,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263,Unf,0,1017,1280,GasA,Ex,Y,SBrkr,1280,0,0,1280,0,0,2,0,2,1,Gd,5,Typ,0,None,Attchd,1992.0,RFn,2,506,TA,TA,Y,0,82,0,0,144,0,None,None,None,0,1,2010,WD,Normal


In [6]:
train_copy.to_pickle(path='data/processed/train.pkl')
test_copy.to_pickle(path='data/processed/test.pkl')

Now we make train/test split

In [8]:
import sys
from sklearn.metrics import mean_squared_error, mean_absolute_error
sys.path.append('..')
from sklearn.model_selection import *
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.pipeline import *
from src.features.features import housing_stock

import catboost

train = pd.read_pickle('data/processed/train.pkl')

def preprocess_before_model(df:pd.DataFrame):
        if (np.sum(list(df[i].isna().sum() > 0 for i in df))):
                df = preprocessing(df)
        encod = LabelEncoder()
        for i in categorial_cols:
                df[i] = encod.fit_transform(df[i])
        df[categorial_cols] = df[categorial_cols].astype('object')
        df = housing_stock(df)

        return df

train = preprocess_before_model(train)

#print(train)
train_x, test_x, train_y, test_y = train_test_split(train, Target, test_size=0.3, random_state=47)

train_x

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,New housing stock
Id,,,,,,,,,,,,,,,,,,,,,
1351,90,3,91.0,11643,1,1,3,3,0,4,...,0,3,4,1,0,8,2009,8,4,0
1409,70,4,60.0,7740,1,1,3,3,0,4,...,0,3,4,1,0,6,2010,8,4,0
277,20,3,129.0,9196,1,1,0,3,0,4,...,0,3,4,1,0,4,2010,8,4,1
418,70,3,86.0,22420,1,1,0,3,0,4,...,0,3,4,1,0,11,2007,8,4,0
687,60,1,84.0,10207,1,1,3,3,0,4,...,0,3,4,1,0,8,2007,6,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,60,3,104.0,21535,1,1,0,3,0,0,...,0,3,4,1,0,1,2007,8,4,1
585,50,4,51.0,6120,1,1,3,3,0,4,...,0,3,4,1,0,7,2009,8,4,0
1289,120,3,40.0,5664,1,1,0,3,0,4,...,0,3,4,1,0,10,2009,8,4,1


I choosed RMSE as a metric for each model, because it shows us how far we from the ideal solution (target) and also penalizes a large prediction error. Also we'll take a look on MAE metrics, which just measures the average magnitude of the errors in a set of forecasts, without considering their direction.

## Pipelined train and inference

In [9]:
pipe = Pipeline(
    [   
        #('preprocess', preprocess_before_model),
        ('LinearRegression', LinearRegression().fit(train_x, train_y).predict(test_x)),
        ('CatBoostRegressor', catboost.CatBoostRegressor(iterations=1000, loss_function='RMSE').fit(train_x, train_y).predict(test_x)),
    ]
)
#pipe.fit(train_x, train_y)
predicted = pipe
#predicted
print(f'LinearRegression RMSE = {mean_squared_error(test_y, predicted[0], squared=False)}\nLinearRegressor MAE = {mean_absolute_error(test_y, predicted[0])}')
print(f'CatBoostRegressor RMSE = {mean_squared_error(test_y, predicted[1], squared=False)}\nCatBoostRegressor MAE = {mean_absolute_error(test_y, predicted[1])}')

Learning rate set to 0.041084
0:	learn: 76901.3458807	total: 154ms	remaining: 2m 33s
1:	learn: 75086.1431551	total: 157ms	remaining: 1m 18s
2:	learn: 73376.1396329	total: 159ms	remaining: 53s
3:	learn: 71440.5193224	total: 162ms	remaining: 40.3s
4:	learn: 69681.0697450	total: 164ms	remaining: 32.7s
5:	learn: 68002.6552033	total: 167ms	remaining: 27.6s
6:	learn: 66277.2617701	total: 169ms	remaining: 24s
7:	learn: 64636.6488657	total: 172ms	remaining: 21.3s
8:	learn: 63214.2019360	total: 174ms	remaining: 19.2s
9:	learn: 61793.1984700	total: 177ms	remaining: 17.5s
10:	learn: 60301.3626117	total: 179ms	remaining: 16.1s
11:	learn: 58877.3476286	total: 182ms	remaining: 15s
12:	learn: 57460.4382654	total: 184ms	remaining: 14s
13:	learn: 56211.3441875	total: 187ms	remaining: 13.2s
14:	learn: 55092.4868588	total: 189ms	remaining: 12.4s
15:	learn: 53950.0446789	total: 192ms	remaining: 11.8s
16:	learn: 52943.1712195	total: 194ms	remaining: 11.2s
17:	learn: 51904.2956133	total: 197ms	remaining: 10

According to RMSE our linear regression model has in average difference between true price and predicted is about 50000\$, meanwhile regression from catboost has significantly lower RMSE (about 33000\$). That means that CatBoostRegressor is the way accurate than linear regression.

If we decide to compare MAE between linear regression and catboost regressor we'll see, that absolute mean error lower in catboost, that means that this model works better.

## Not pipelined train and inference

In [10]:
linear_regression = LinearRegression()
linear_regression.fit(train_x, train_y)
predicted_linreg = linear_regression.predict(test_x)
print(f'LinearRegression RMSE = {mean_squared_error(test_y, predicted_linreg, squared=False)}\nLinearRegressor MAE = {mean_absolute_error(test_y, predicted_linreg)}')

LinearRegression RMSE = 50120.17961840389
LinearRegressor MAE = 21840.399475152964


In [11]:
train_dataset = catboost.Pool(train_x, train_y)
test_dataset = catboost.Pool(test_x, test_y)

catboost_regression = catboost.CatBoostRegressor(iterations=1000, loss_function='RMSE')
catboost_regression.fit(train_x, train_y)
predicted_cbreg = catboost_regression.predict(test_x)
print(f'CatBoostRegressor RMSE = {mean_squared_error(test_y, predicted_cbreg, squared=False)}\nCatBoostRegressor MAE = {mean_absolute_error(test_y, predicted_cbreg)}')

Learning rate set to 0.041084
0:	learn: 76901.3458807	total: 4.1ms	remaining: 4.09s
1:	learn: 75086.1431551	total: 7.11ms	remaining: 3.55s
2:	learn: 73376.1396329	total: 10.6ms	remaining: 3.53s
3:	learn: 71440.5193224	total: 13.1ms	remaining: 3.27s
4:	learn: 69681.0697450	total: 16.3ms	remaining: 3.24s
5:	learn: 68002.6552033	total: 19.7ms	remaining: 3.26s
6:	learn: 66277.2617701	total: 23.3ms	remaining: 3.31s
7:	learn: 64636.6488657	total: 26.4ms	remaining: 3.27s
8:	learn: 63214.2019360	total: 29.5ms	remaining: 3.25s
9:	learn: 61793.1984700	total: 32.2ms	remaining: 3.19s
10:	learn: 60301.3626117	total: 34.9ms	remaining: 3.13s
11:	learn: 58877.3476286	total: 37.5ms	remaining: 3.08s
12:	learn: 57460.4382654	total: 40ms	remaining: 3.04s
13:	learn: 56211.3441875	total: 42.6ms	remaining: 3s
14:	learn: 55092.4868588	total: 45.4ms	remaining: 2.98s
15:	learn: 53950.0446789	total: 47.8ms	remaining: 2.94s
16:	learn: 52943.1712195	total: 50.5ms	remaining: 2.92s
17:	learn: 51904.2956133	total: 53